In [1]:
# General
# %pip install numpy
# %pip install scipy
# %pip install matplotlib
# %pip install soundfile
import numpy as np

# Signal Processing
from scipy import signal
from scipy.fftpack import fft, ifft

# Audio IO
from IPython.display import Audio 
import soundfile as sf

# Plotting
import matplotlib.pyplot as plt

In [2]:
x, Fs = sf.read("bad_audio.wav")

Audio(x, rate = Fs)

In [ ]:
# implement 2nd order peaking filter to remove 573Hz tone here

hp_audio = np.array.zeros(x.length())
fC = 574
fB = 1

G = -.5
# c = (np.tan(np.pi * wB/2)-v0) / (np.tan(np.pi * wB/2)+v0)
# d = -np.cos(np.pi * wC)
# xH = [0,0]

def second_order_peak(audio_arr, cutoff, bandwidth, sr, G):
    wC = 2 * cutoff/sr
    wB = 2 * bandwidth/sr   
    v0 = 10 ** (G / 20)
    h0 = v0 - 1
    c = None
    if G >= 0:
        c = (np.tan(np.pi * wB/2)-1) / (np.tan(np.pi * wB/2)+1)
    else:
        c = (np.tan(np.pi * wB/2)-v0) / (np.tan(np.pi * wB/2)+v0)
    d = -np.cos(np.pi * wC)
    xH = [0,0]
    for i in range(len(audio_arr)):
        xH[0] = audio_arr[i] - c * xH[1]
        y = h0 * xH[0] - d * xH[1]
        xH[1] = audio_arr[i] - c * xH[0]
        audio_arr[i] = y



K = np.tan(np.pi * fC / Fs)

G = 1

h0 = v0 - 1
# d = -np.cos(2 * np.pi * fC / Fs)
cB = (np.tan(np.pi * fB / Fs) - 1) / (np.tan(np.pi * fB / Fs) + 1)
cC = (np.tan(np.pi * fB / Fs) - K) / (np.tan(np.pi * fB / Fs) + K)
# Coeffecients for LF cut
b0 = (v0 * (1 + np.sqrt(2) * K + K ** 2)) / (v0 + np.sqrt(2 * v0) * K + K ** 2)
b1 = (2 * (v0 * (K ** 2 - 1))) / (v0 + np.sqrt(2 * v0) * K + K ** 2)
b2 = (v0 * (1 - np.sqrt(2) * K + K ** 2)) / (v0 + np.sqrt(2 * v0) * K + K ** 2)
a1 = (2 * (K ** 2 - 1)) / (1 + np.sqrt(2) * K + K ** 2)
# a2 = (v0 - np.sqrt(2 * v0) * K + K ** 2) / (v0 + np.sqrt(2 * v0) * K + K ** 2)


# 2nd order peak filter
a2 = -cB + d(1 - )
Hz = 1 + (h0/2)[1 - a2(z)]

In [4]:
# implement 2nd order high shelf filter to remove >14kHz noise here



In [5]:
# Audio(y, rate = Fs)